In [261]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import glob

In [262]:
# read in the data frame, drop the unnecessary index row & transpose it so rows are loci and columns are individuals.
df = pd.read_csv("~/Documents/UMontana/Research/YNP/AHQsd/AHQsd_analyses/AHQsd_F2_genotypes_raw.csv")

In [263]:
df.set_index('Unnamed: 0')

,1_12041,1_12077,1_15991,1_15995,1_16002,1_16071,1_16073,1_16111,1_16175,1_23477,...,14_26113525,14_26113560,14_26113569,14_26113586,14_26117881,14_26117882,14_26117958,14_26117961,14_26117991,14_26118001
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
/home/colette_berg/AHQsd_2021/AHQsd5.1/processed/AHQsd_5.01A,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd5.1/processed/AHQsd_5.01B,-1,-1,0,0,0,0,0,-1,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd5.1/processed/AHQsd_5.01C,1,1,0,0,0,1,1,2,0,0,...,2,2,2,2,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd5.1/processed/AHQsd_5.01D,0,0,0,0,0,0,0,-1,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd5.1/processed/AHQsd_5.01E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/home/colette_berg/AHQsd_2021/AHQsd_3.2/processed/AHQsd_3.12D,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd_3.2/processed/AHQsd_3.12E,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
/home/colette_berg/AHQsd_2021/AHQsd_3.2/processed/AHQsd_3.12F,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [264]:
tdf = df.set_index('Unnamed: 0').T.reset_index()

In [265]:
# label by what linkage group the loci belong to
tmpDF = pd.DataFrame(columns=['chr','site'])
tmpDF[['chr','site']] = tdf['index'].str.split('_', expand=True)


In [266]:
tdf['chr'] = tmpDF['chr']
tdf['site'] = tmpDF['site']

In [268]:
# replace -1 with nans

for i in tdf.columns:
    tdf[i].replace(-1,np.nan,inplace=True)

In [269]:
# split them
for chromo in range(1,15):
    lg_chromo = tdf[tdf.chr == str(chromo)]
    lg_chromo.to_csv(f'data/lg_chromo_{chromo}.csv',index=False)

In [348]:
def list_name_indivs(tdf):
    return tdf.columns.to_list()[1:385] 

def create_bins(subset_ds):
    # create bins of 18 SNPs each in each linkage group
    bins = [i for i in np.arange(subset_ds.index.min(), subset_ds.index.max(), 25)] + [np.inf]
    return bins

def create_stats_template(ds, chromo_number):
    # create dataframes for the loop to write to. 
    return pd.DataFrame(columns=["indv_name","count","mean",f"num_of_nans_chr{chromo_number}"])

def cut_by_bins(subset_ds, bins):
    return pd.cut(subset_ds.index,bins=bins)

def create_stats_dataframe(small_list, ds, bins, chromo_number):
    stats_dataframe = create_stats_template(ds, chromo_number)
    chromo_number = ds['chr'].iloc[0]
    for col in small_list:
        mdf = ds[['index',col]]
        grouped_df = mdf.groupby(bins)[col].agg(['mean','count'])
        grouped_df[f"num_of_nans_chr{chromo_number}"] = 25 - grouped_df["count"]
        grouped_df["indv_name"] = col
        stats_dataframe  = stats_dataframe.append(grouped_df)
    return stats_dataframe

def remove_high_NaNs(stats_dataframe, chromo_number):
    stats_dataframe.loc[stats_dataframe[f"num_of_nans_chr{chromo_number}"] > 17, 'mean'] = np.nan
    return stats_dataframe

def narrow(filt_stats_dataframe): 
    filt_stats_narrow = filt_stats_dataframe[['indv_name', 'mean']]
    filt_stats_narrow['bin'] = filt_stats_narrow.index
    filt_stats_wide = filt_stats_narrow.pivot_table(index='bin', columns='indv_name', values='mean', aggfunc='first', dropna=False)
    return filt_stats_wide

def cleanup_cols_names(df):
    df.columns = df.columns.str.split('processed/').str[1].tolist()   
    return df

def create_bin_template():
    col_names = ['level_0', 'first','last']
    return pd.DataFrame(columns=col_names)

def create_append_template():
    col_names = ['AHQsd_5.01A',
     'AHQsd_5.01B',
     'AHQsd_5.01C',
     'AHQsd_5.01D',
     'AHQsd_5.01E',
     'AHQsd_5.01F',
     'AHQsd_5.01G',
     'AHQsd_5.01H',
     'AHQsd_5.02A',
     'AHQsd_5.02B',
     'AHQsd_5.02C',
     'AHQsd_5.02D',
     'AHQsd_5.02E',
     'AHQsd_5.02F',
     'AHQsd_5.02G',
     'AHQsd_5.02H',
     'AHQsd_5.03A',
     'AHQsd_5.03B',
     'AHQsd_5.03C',
     'AHQsd_5.03D',
     'AHQsd_5.03E',
     'AHQsd_5.03F',
     'AHQsd_5.03G',
     'AHQsd_5.03H',
     'AHQsd_5.04A',
     'AHQsd_5.04B',
     'AHQsd_5.04C',
     'AHQsd_5.04D',
     'AHQsd_5.04E',
     'AHQsd_5.04F',
     'AHQsd_5.04G',
     'AHQsd_5.04H',
     'AHQsd_5.05A',
     'AHQsd_5.05B',
     'AHQsd_5.05C',
     'AHQsd_5.05D',
     'AHQsd_5.05E',
     'AHQsd_5.05F',
     'AHQsd_5.05G',
     'AHQsd_5.05H',
     'AHQsd_5.06A',
     'AHQsd_5.06B',
     'AHQsd_5.06C',
     'AHQsd_5.06D',
     'AHQsd_5.06E',
     'AHQsd_5.06F',
     'AHQsd_5.06G',
     'AHQsd_5.06H',
     'AHQsd_5.07A',
     'AHQsd_5.07B',
     'AHQsd_5.07C',
     'AHQsd_5.07D',
     'AHQsd_5.07E',
     'AHQsd_5.07F',
     'AHQsd_5.07G',
     'AHQsd_5.07H',
     'AHQsd_5.08A',
     'AHQsd_5.08B',
     'AHQsd_5.08C',
     'AHQsd_5.08D',
     'AHQsd_5.08E',
     'AHQsd_5.08F',
     'AHQsd_5.08G',
     'AHQsd_5.08H',
     'AHQsd_5.09A',
     'AHQsd_5.09B',
     'AHQsd_5.09C',
     'AHQsd_5.09D',
     'AHQsd_5.09E',
     'AHQsd_5.09F',
     'AHQsd_5.09G',
     'AHQsd_5.09H',
     'AHQsd_5.10A',
     'AHQsd_5.10B',
     'AHQsd_5.10C',
     'AHQsd_5.10D',
     'AHQsd_5.10E',
     'AHQsd_5.10F',
     'AHQsd_5.10G',
     'AHQsd_5.10H',
     'AHQsd_5.11A',
     'AHQsd_5.11B',
     'AHQsd_5.11C',
     'AHQsd_5.11D',
     'AHQsd_5.11E',
     'AHQsd_5.11F',
     'AHQsd_5.11G',
     'AHQsd_5.11H',
     'AHQsd_5.12A',
     'AHQsd_5.12B',
     'AHQsd_5.12C',
     'AHQsd_5.12D',
     'AHQsd_5.12E',
     'AHQsd_5.12F',
     'AHQsd_5.12G',
     'AHQsd_5.12H',
     'AHQsd_6.01A',
     'AHQsd_6.01B',
     'AHQsd_6.01C',
     'AHQsd_6.01D',
     'AHQsd_6.01E',
     'AHQsd_6.01F',
     'AHQsd_6.01G',
     'AHQsd_6.01H',
     'AHQsd_6.02A',
     'AHQsd_6.02B',
     'AHQsd_6.02C',
     'AHQsd_6.02D',
     'AHQsd_6.02E',
     'AHQsd_6.02F',
     'AHQsd_6.02G',
     'AHQsd_6.02H',
     'AHQsd_6.03A',
     'AHQsd_6.03B',
     'AHQsd_6.03C',
     'AHQsd_6.03D',
     'AHQsd_6.03E',
     'AHQsd_6.03F',
     'AHQsd_6.03G',
     'AHQsd_6.03H',
     'AHQsd_6.04A',
     'AHQsd_6.04B',
     'AHQsd_6.04C',
     'AHQsd_6.04D',
     'AHQsd_6.04E',
     'AHQsd_6.04F',
     'AHQsd_6.04G',
     'AHQsd_6.04H',
     'AHQsd_6.05A',
     'AHQsd_6.05B',
     'AHQsd_6.05C',
     'AHQsd_6.05D',
     'AHQsd_6.05E',
     'AHQsd_6.05F',
     'AHQsd_6.05G',
     'AHQsd_6.05H',
     'AHQsd_6.06A',
     'AHQsd_6.06B',
     'AHQsd_6.06C',
     'AHQsd_6.06D',
     'AHQsd_6.06E',
     'AHQsd_6.06F',
     'AHQsd_6.06G',
     'AHQsd_6.06H',
     'AHQsd_6.07A',
     'AHQsd_6.07B',
     'AHQsd_6.07C',
     'AHQsd_6.07D',
     'AHQsd_6.07E',
     'AHQsd_6.07F',
     'AHQsd_6.07G',
     'AHQsd_6.07H',
     'AHQsd_6.08A',
     'AHQsd_6.08B',
     'AHQsd_6.08C',
     'AHQsd_6.08D',
     'AHQsd_6.08E',
     'AHQsd_6.08F',
     'AHQsd_6.08G',
     'AHQsd_6.08H',
     'AHQsd_6.09A',
     'AHQsd_6.09B',
     'AHQsd_6.09C',
     'AHQsd_6.09D',
     'AHQsd_6.09E',
     'AHQsd_6.09F',
     'AHQsd_6.09G',
     'AHQsd_6.09H',
     'AHQsd_6.10A',
     'AHQsd_6.10B',
     'AHQsd_6.10C',
     'AHQsd_6.10D',
     'AHQsd_6.10E',
     'AHQsd_6.10F',
     'AHQsd_6.10G',
     'AHQsd_6.10H',
     'AHQsd_6.11A',
     'AHQsd_6.11B',
     'AHQsd_6.11C',
     'AHQsd_6.11D',
     'AHQsd_6.11E',
     'AHQsd_6.11F',
     'AHQsd_6.11G',
     'AHQsd_6.11H',
     'AHQsd_6.12A',
     'AHQsd_6.12B',
     'AHQsd_6.12C',
     'AHQsd_6.12D',
     'AHQsd_6.12E',
     'AHQsd_6.12F',
     'AHQsd_6.12G',
     'AHQsd_6.12H',
     'AHQsd_1.01A',
     'AHQsd_1.01B',
     'AHQsd_1.01C',
     'AHQsd_1.01D',
     'AHQsd_1.01E',
     'AHQsd_1.01F',
     'AHQsd_1.01G',
     'AHQsd_1.01H',
     'AHQsd_1.02A',
     'AHQsd_1.02B',
     'AHQsd_1.02C',
     'AHQsd_1.02D',
     'AHQsd_1.02E',
     'AHQsd_1.02F',
     'AHQsd_1.02G',
     'AHQsd_1.02H',
     'AHQsd_1.03A',
     'AHQsd_1.03B',
     'AHQsd_1.03C',
     'AHQsd_1.03D',
     'AHQsd_1.03E',
     'AHQsd_1.03F',
     'AHQsd_1.03G',
     'AHQsd_1.03H',
     'AHQsd_1.04A',
     'AHQsd_1.04B',
     'AHQsd_1.04C',
     'AHQsd_1.04D',
     'AHQsd_1.04E',
     'AHQsd_1.04F',
     'AHQsd_1.04G',
     'AHQsd_1.04H',
     'AHQsd_1.05A',
     'AHQsd_1.05B',
     'AHQsd_1.05C',
     'AHQsd_1.05D',
     'AHQsd_1.05E',
     'AHQsd_1.05F',
     'AHQsd_1.05G',
     'AHQsd_1.05H',
     'AHQsd_1.06A',
     'AHQsd_1.06B',
     'AHQsd_1.06C',
     'AHQsd_1.06D',
     'AHQsd_1.06E',
     'AHQsd_1.06F',
     'AHQsd_1.06G',
     'AHQsd_1.06H',
     'AHQsd_1.07A',
     'AHQsd_1.07B',
     'AHQsd_1.07C',
     'AHQsd_1.07D',
     'AHQsd_1.07E',
     'AHQsd_1.07F',
     'AHQsd_1.07G',
     'AHQsd_1.07H',
     'AHQsd_1.08A',
     'AHQsd_1.08B',
     'AHQsd_1.08C',
     'AHQsd_1.08D',
     'AHQsd_1.08E',
     'AHQsd_1.08F',
     'AHQsd_1.08G',
     'AHQsd_1.08H',
     'AHQsd_1.09A',
     'AHQsd_1.09B',
     'AHQsd_1.09C',
     'AHQsd_1.09D',
     'AHQsd_1.09E',
     'AHQsd_1.09F',
     'AHQsd_1.09G',
     'AHQsd_1.09H',
     'AHQsd_1.10A',
     'AHQsd_1.10B',
     'AHQsd_1.10C',
     'AHQsd_1.10D',
     'AHQsd_1.10E',
     'AHQsd_1.10F',
     'AHQsd_1.10G',
     'AHQsd_1.10H',
     'AHQsd_1.11A',
     'AHQsd_1.11B',
     'AHQsd_1.11C',
     'AHQsd_1.11D',
     'AHQsd_1.11E',
     'AHQsd_1.11F',
     'AHQsd_1.11G',
     'AHQsd_1.11H',
     'AHQsd_1.12A',
     'AHQsd_1.12B',
     'AHQsd_1.12C',
     'AHQsd_1.12D',
     'AHQsd_1.12E',
     'AHQsd_1.12F',
     'AHQsd_1.12G',
     'AHQsd_1.12H',
     'AHQsd_3.01A',
     'AHQsd_3.01B',
     'AHQsd_3.01C',
     'AHQsd_3.01D',
     'AHQsd_3.01E',
     'AHQsd_3.01F',
     'AHQsd_3.01G',
     'AHQsd_3.01H',
     'AHQsd_3.02A',
     'AHQsd_3.02B',
     'AHQsd_3.02C',
     'AHQsd_3.02D',
     'AHQsd_3.02E',
     'AHQsd_3.02F',
     'AHQsd_3.02G',
     'AHQsd_3.02H',
     'AHQsd_3.03A',
     'AHQsd_3.03B',
     'AHQsd_3.03C',
     'AHQsd_3.03D',
     'AHQsd_3.03E',
     'AHQsd_3.03F',
     'AHQsd_3.03G',
     'AHQsd_3.03H',
     'AHQsd_3.04A',
     'AHQsd_3.04B',
     'AHQsd_3.04C',
     'AHQsd_3.04D',
     'AHQsd_3.04E',
     'AHQsd_3.04F',
     'AHQsd_3.04G',
     'AHQsd_3.04H',
     'AHQsd_3.05A',
     'AHQsd_3.05B',
     'AHQsd_3.05C',
     'AHQsd_3.05D',
     'AHQsd_3.05E',
     'AHQsd_3.05F',
     'AHQsd_3.05G',
     'AHQsd_3.05H',
     'AHQsd_3.06A',
     'AHQsd_3.06B',
     'AHQsd_3.06C',
     'AHQsd_3.06D',
     'AHQsd_3.06E',
     'AHQsd_3.06F',
     'AHQsd_3.06G',
     'AHQsd_3.06H',
     'AHQsd_3.07A',
     'AHQsd_3.07B',
     'AHQsd_3.07C',
     'AHQsd_3.07D',
     'AHQsd_3.07E',
     'AHQsd_3.07F',
     'AHQsd_3.07G',
     'AHQsd_3.07H',
     'AHQsd_3.08A',
     'AHQsd_3.08B',
     'AHQsd_3.08C',
     'AHQsd_3.08D',
     'AHQsd_3.08E',
     'AHQsd_3.08F',
     'AHQsd_3.08G',
     'AHQsd_3.08H',
     'AHQsd_3.09A',
     'AHQsd_3.09B',
     'AHQsd_3.09C',
     'AHQsd_3.09D',
     'AHQsd_3.09E',
     'AHQsd_3.09F',
     'AHQsd_3.09G',
     'AHQsd_3.09H',
     'AHQsd_3.10A',
     'AHQsd_3.10B',
     'AHQsd_3.10C',
     'AHQsd_3.10D',
     'AHQsd_3.10E',
     'AHQsd_3.10F',
     'AHQsd_3.10G',
     'AHQsd_3.10H',
     'AHQsd_3.11A',
     'AHQsd_3.11B',
     'AHQsd_3.11C',
     'AHQsd_3.11D',
     'AHQsd_3.11E',
     'AHQsd_3.11F',
     'AHQsd_3.11G',
     'AHQsd_3.11H',
     'AHQsd_3.12A',
     'AHQsd_3.12B',
     'AHQsd_3.12C',
     'AHQsd_3.12D',
     'AHQsd_3.12E',
     'AHQsd_3.12F',
     'AHQsd_3.12G',
     'AHQsd_3.12H']
    return pd.DataFrame(columns=col_names)

def convert_to_genotypes(template_df):
    choiceList = ['BB', 'AA', 'AB']
    all_indvs = template_df.columns
    for i in all_indvs:
        condList = [(template_df[i] > 1.8), (template_df[i] < .2), ((template_df[i] < 1.6) & (template_df[i] > .4))]
        template_df[i] = np.select(condList, choiceList)
    for i in template_df.columns:
        template_df[i].replace('0',np.nan,inplace=True)
    return template_df

def rename_SNP_bins(ds):
    bin_rename = pd.DataFrame(ds['index'].groupby(cut_bins).agg(['first', 'last']).stack())
    bin_rename = bin_rename.reset_index()
    bin_rename = bin_rename.pivot_table(index='level_0', columns='level_1', values =0,aggfunc='first', dropna=False)
    bin_rename = bin_rename.reset_index()
    return bin_rename

#bin_1 = pd.DataFrame(lg1['index'].groupby(cut_bins1).agg(['first', 'last']).stack())
#bin_1 = bin_1.reset_index()
#bin_1_1 = bin_1.pivot_table(index='level_0', columns='level_1', values =0,aggfunc='first', dropna=False)
#bin_1_1.reset_index()

In [349]:
SNP_bin_rename.columns

Index(['level_0', 'first', 'last'], dtype='object', name='level_1')

In [361]:
subset_file_list = glob.glob('data/*.csv')
ind_list = list_name_indivs(tdf)

template_df = create_append_template()
bin_df = create_bin_template()

for fil in subset_file_list:
    print(fil)
    #load subset of dataset that was subset by chromosome
    ds = pd.read_csv(fil)
    chromo_number = ds['chr'].iloc[0]
    #create bins
    bins = create_bins(ds)
    #create empty stats template
    stats_template_df = create_stats_template(ds, chromo_number)
    # cut each linkage group into bins of X SNPs each.
    cut_bins = cut_by_bins(ds, bins)
    # start renaming SNP bins
    SNP_bin_rename = rename_SNP_bins(ds)
    # create dataframe of stats
    stats_dataframe = create_stats_dataframe(ind_list, ds, cut_bins, chromo_number)
    # remove dfs with lots of NAs 
    filt_stats_dataframe = remove_high_NaNs(stats_dataframe, chromo_number)
    # pivot magic
    filt_stats_wide = narrow(filt_stats_dataframe)
    #cleanup col names
    clean_df = cleanup_cols_names(filt_stats_wide)
    template_df = template_df.append(clean_df,ignore_index=True)
    bin_df = bin_df.append(SNP_bin_rename, ignore_index=True)
#     break
#     cleaned_ds.to_csv('final_data')

data/lg_chromo_9.csv


<ipython-input-348-4d5951b3c544>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_stats_narrow['bin'] = filt_stats_narrow.index


data/lg_chromo_8.csv
data/lg_chromo_5.csv
data/lg_chromo_14.csv
data/lg_chromo_4.csv
data/lg_chromo_6.csv
data/lg_chromo_7.csv
data/lg_chromo_3.csv
data/lg_chromo_12.csv
data/lg_chromo_13.csv
data/lg_chromo_2.csv
data/lg_chromo_11.csv
data/lg_chromo_10.csv
data/lg_chromo_1.csv


In [364]:
# label the bins by the means of their SNP positions

tmpDF1 = pd.DataFrame(columns=['chr','site1', 'chr_', 'site2'])
tmpDF1[['chr','site1']] = bin_df['first'].str.split('_', expand=True)
tmpDF1[['chr_','site2']] = bin_df['last'].str.split('_', expand=True)

tmpDF1['sum'] = pd.to_numeric(tmpDF1['site1']) + pd.to_numeric(tmpDF1['site2'])

tmpDF1['mean'] = tmpDF1['sum'] / 2

tmpDF1['bin_mean'] = tmpDF1['chr']+"_"+tmpDF1['mean'].astype(str)

#add the means to the whole spreadsheet
template_df.insert(loc=0, column='bin_mean', value=tmpDF1['bin_mean'])


In [366]:
# add a chr column
template_df.insert(loc=0, column='chr', value=tmpDF1['chr'])

In [368]:
# add a pos column
template_df.insert(loc=0, column='pos', value=tmpDF1['mean'])

In [40]:
#write to csv
template_df.to_csv("AHQsd_F2_SNPs_windowed.csv")